In [29]:
from mysql.connector import pooling
from concurrent.futures import ThreadPoolExecutor

In [30]:
dbconfig = {
    'host': 'localhost',
    'port': '19950',
    'user': 'root',
    'password': 'dsjk@186115!',
    'database': 'amp',
    'charset': 'utf8mb4'
}
cnxpool = pooling.MySQLConnectionPool(pool_name="mypool",
                                      pool_size=10,  # The pool size. If this argument is not given, the default is 5.
                                      **dbconfig)  # 创建线程池

In [31]:
def execute_query(query):
    conn = cnxpool.get_connection()  # Gets a connection from the pool.
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    conn.close()  # Do not close, but adds connection back to pool.
    return results, column_names


queries = [
    "SELECT id, city_name FROM ai_setting limit 1;",
    "SELECT * FROM ai_bca limit 1;",
    "SELECT * FROM ai_kbextra limit 1;"
]
with ThreadPoolExecutor(max_workers=cnxpool.pool_size) as executor:
    f1_result = [executor.submit(execute_query, query) for query in queries]
    print([f1r.result() for f1r in f1_result])


[([(1, '深圳市')], ['id', 'city_name']), ([(1, '保持心情平和：保持心情平和可以缓解压力，促进身体健康。可以通过娱乐、旅游、交友等方式来调节情绪。', '阴性', datetime.datetime(2024, 7, 12, 9, 59, 25))], ['bid', 'context', 'diag_name', 'update_time']), ([(306, 2499, '*', '鳞状上皮细胞癌抗原偏高', '', '', 0, 0, '轻度', '5.00', '19.99', 0, 4, 0, 199, 0, '建议1月后复查，如复查仍偏高请专科就诊', '鳞状细胞癌相关抗原(SCC)是一种较好的鳞状细胞癌标志物。SCC对宫颈、肺、食管和头颈部鳞癌的诊断、疗效监测、复发监测以及评价预后有较大意义。SCC是最早用于诊断鳞癌的肿瘤标志物，患子宫颈癌、鼻咽癌、肺鳞癌、食管癌时，血清SCC升高。一些良性疾病，如肝炎、肝硬化、胰腺炎、肺炎、结核、湿疹、银屑病等， SCC也有一定程度的升高。肾功能障碍也会引起SCC水平升高。', datetime.datetime(2025, 4, 18, 17, 19, 22), 'lijingrong@183.14.30.199', 1)], ['eid', 'tid', 'use_by', 't_name', 'normal_name', 'main_name', 'locked', 'doc_id', 'e_degree', 'e_from', 'e_to', 'fixed_value', 'e_classify', 'age_from', 'age_to', 'e_gender', 'e_advice', 'e_explain', 'updatetime', 'updated_by', 'tmptype'])]
